In [1]:
import pandas as pd
import re
import nltk

In [2]:
df_train = pd.read_csv("/home/devjith/Luminar/Machine Learning/files/SMS_train.csv",encoding='ISO-8859-1')
df_test = pd.read_csv("/home/devjith/Luminar/Machine Learning/files/SMS_test.csv",encoding='ISO-8859-1')

In [3]:
df_test

,S. No.,Message_body,Label
0,1,"UpgrdCentre Orange customer, you may now claim...",Spam
1,2,"Loan for any purpose £500 - £75,000. Homeowner...",Spam
2,3,Congrats! Nokia 3650 video camera phone is you...,Spam
3,4,URGENT! Your Mobile number has been awarded wi...,Spam
4,5,Someone has contacted our dating service and e...,Spam
...,...,...,...
120,121,7 wonders in My WORLD 7th You 6th Ur style 5th...,Non-Spam
121,122,Try to do something dear. You read something f...,Non-Spam
122,123,Sun ah... Thk mayb can if dun have anythin on....,Non-Spam
123,124,"SYMPTOMS when U are in love: ""1.U like listeni...",Non-Spam


In [4]:
df = pd.concat([df_train,df_test],axis=0,ignore_index=True)

In [5]:
df.reset_index

<bound method DataFrame.reset_index of       S. No.                                       Message_body     Label
0          1                         Rofl. Its true to its name  Non-Spam
1          2  The guy did some bitching but I acted like i'd...  Non-Spam
2          3  Pity, * was in mood for that. So...any other s...  Non-Spam
3          4               Will ü b going to esplanade fr home?  Non-Spam
4          5  This is the 2nd time we have tried 2 contact u...      Spam
...      ...                                                ...       ...
1077     121  7 wonders in My WORLD 7th You 6th Ur style 5th...  Non-Spam
1078     122  Try to do something dear. You read something f...  Non-Spam
1079     123  Sun ah... Thk mayb can if dun have anythin on....  Non-Spam
1080     124  SYMPTOMS when U are in love: "1.U like listeni...  Non-Spam
1081     125  Great. Have a safe trip. Dont panic surrender ...  Non-Spam

[1082 rows x 3 columns]>

In [6]:
df = df.drop(columns='S. No.')

In [7]:
df

,Message_body,Label
0,Rofl. Its true to its name,Non-Spam
1,The guy did some bitching but I acted like i'd...,Non-Spam
2,"Pity, * was in mood for that. So...any other s...",Non-Spam
3,Will ü b going to esplanade fr home?,Non-Spam
4,This is the 2nd time we have tried 2 contact u...,Spam
...,...,...
1077,7 wonders in My WORLD 7th You 6th Ur style 5th...,Non-Spam
1078,Try to do something dear. You read something f...,Non-Spam
1079,Sun ah... Thk mayb can if dun have anythin on....,Non-Spam
1080,"SYMPTOMS when U are in love: ""1.U like listeni...",Non-Spam


In [8]:
df['Label'] = df['Label'].map({'Non-Spam':0,'Spam':1})

In [9]:
df

,Message_body,Label
0,Rofl. Its true to its name,0
1,The guy did some bitching but I acted like i'd...,0
2,"Pity, * was in mood for that. So...any other s...",0
3,Will ü b going to esplanade fr home?,0
4,This is the 2nd time we have tried 2 contact u...,1
...,...,...
1077,7 wonders in My WORLD 7th You 6th Ur style 5th...,0
1078,Try to do something dear. You read something f...,0
1079,Sun ah... Thk mayb can if dun have anythin on....,0
1080,"SYMPTOMS when U are in love: ""1.U like listeni...",0


In [10]:
text = df['Message_body']

In [11]:
from nltk.tokenize import TweetTokenizer,word_tokenize
tk = TweetTokenizer()

text = text.apply(lambda x:tk.tokenize(x)).apply(lambda x:' '.join(x) )

In [12]:
text = text.apply(lambda x:re.sub('[^a-zA-Z0-9]',' ',x))

In [13]:
text = text.apply(lambda x: ' '.join([i for i in word_tokenize(x) if len(i)>=3]))

In [14]:
from nltk.stem import SnowballStemmer

stemmer = SnowballStemmer(language='english')

text = text.apply(lambda x: ' '.join([stemmer.stem(i.lower()) for i in word_tokenize(x)]))

In [15]:
from nltk.corpus import stopwords
stop = stopwords.words('english')

text = text.apply(lambda x:' '.join([i for i in word_tokenize(x) if i not in stop]))

In [16]:
text

0                                          rofl true name
1       guy bitch act like interest buy someth els nex...
2                                   piti mood ani suggest
3                                        go esplanad home
4       2nd time tri contact 750 pound prize claim eas...
                              ...                        
1077    wonder world 7th 6th style 5th smile 4th perso...
1078                     tri someth dear read someth exam
1079    sun thk mayb dun anythin thk book lesson pilat...
1080    symptom love like listen song get stop see nam...
1081                   great safe trip dont panic surrend
Name: Message_body, Length: 1082, dtype: object

In [17]:
from sklearn.feature_extraction.text import TfidfVectorizer

vec = TfidfVectorizer()

train_data = vec.fit_transform(text)
train_data

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 8922 stored elements and shape (1082, 2852)>

In [18]:
y = df['Label']

In [19]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(train_data,y,test_size=0.3,random_state=42)

In [20]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier


knn = KNeighborsClassifier(n_neighbors=7)
naive = MultinomialNB()
svm = SVC()
decision = DecisionTreeClassifier(criterion='entropy')
random = RandomForestClassifier(n_estimators=1000,criterion='entropy',n_jobs=-1)


In [21]:
models = [knn,naive,svm,decision,random]
from sklearn.metrics import accuracy_score
for i in models:
    i.fit(x_train,y_train)
    print(f"Accuracy of {i} is {accuracy_score(y_test,i.predict(x_test))}")
    


Accuracy of KNeighborsClassifier(n_neighbors=7) is 0.8338461538461538
Accuracy of MultinomialNB() is 0.9353846153846154
Accuracy of SVC() is 0.9230769230769231
Accuracy of DecisionTreeClassifier(criterion='entropy') is 0.9415384615384615
Accuracy of RandomForestClassifier(criterion='entropy', n_estimators=1000, n_jobs=-1) is 0.96


In [ ]:
sms = " congratulation you have won 50cr please click 'http://abcd.xyx.com' adn fill the form to claim your price "

sms = ' '.join(word_tokenize(sms))

sms = ' '.join([i for i in word_tokenize(sms) if len(i)>=3])
sms = ' '.join([stemmer.stem(i.lower()) for i in word_tokenize(sms)])
sms = ' '.join([i for i in word_tokenize(sms) if i not in sms])

sms = vec.transform([sms])

In [27]:
random.predict(sms)

array([0])